In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
df = pd.read_parquet(Path("data/03_primary/mega_id_labels.parquet"))
df.head()

In [ ]:
df.info()

In [ ]:
df.groupby("is_repeat_buyer").agg({"customer_unique_id": "nunique"})

In [ ]:
features = pd.read_parquet(Path("data/04_feature/transaction_features.parquet"))
features.info()

In [ ]:
features.groupby("is_repeat_buyer").agg({"customer_unique_id": "nunique"})

In [ ]:
features[features.order_id == "1d251ab94983c4adb11e4b168abb1439"].T